In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
import os
os.chdir('C:\\code\\activ')

In [3]:
df = pd.read_excel('./documents/aug_nine_var.xlsx').iloc[:,1::]
X = df.iloc[:,0:21]
y = df.iloc[:,21::].to_numpy()

In [4]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
X.shape

(395, 21)

In [5]:
dnn_model = tf.keras.models.load_model('./model/dnn.h5')

In [6]:
def my_loss(real, X_pred):
    y_pred = dnn_model(X_pred)
    return tf.math.subtract(y_pred, 42)

In [7]:
generator = tf.keras.Sequential()
generator.add(tf.keras.layers.Dense(128, input_dim=21, activation='relu'))
generator.add(tf.keras.layers.Dense(64, activation='relu'))
generator.add(tf.keras.layers.Dense(32, activation='relu'))
generator.add(tf.keras.layers.Dense(16, activation='relu'))
generator.add(tf.keras.layers.Dense(8, activation='relu'))
generator.add(tf.keras.layers.Dense(4, activation='relu'))
generator.add(tf.keras.layers.Dense(21, activation='sigmoid'))

generator.compile(loss=my_loss, optimizer='adam')

In [8]:
result = []
pop_result = []
for i in range(1000):
    noise = tf.random.normal([1, 21])
    X_pred = generator.predict(noise, verbose=0)
    loss = my_loss(noise, X_pred)

    pop = dnn_model.predict(X_pred, verbose=0)
    generator.train_on_batch(noise, X_pred)
    
    result.append(X_pred)
    pop_result.append(pop)

    print(f"epoch: {i}, loss: {loss}, pop: {pop}")

epoch: 0, loss: [[-0.9946861]], pop: [[41.005314]]
epoch: 1, loss: [[-0.9764404]], pop: [[41.02356]]
epoch: 2, loss: [[-1.0225372]], pop: [[40.977463]]
epoch: 3, loss: [[-0.95490646]], pop: [[41.045094]]
epoch: 4, loss: [[-1.0584145]], pop: [[40.941586]]
epoch: 5, loss: [[-1.0767097]], pop: [[40.92329]]
epoch: 6, loss: [[-1.098568]], pop: [[40.901432]]
epoch: 7, loss: [[-1.1327629]], pop: [[40.867237]]
epoch: 8, loss: [[-1.1035309]], pop: [[40.89647]]
epoch: 9, loss: [[-1.1249161]], pop: [[40.875084]]
epoch: 10, loss: [[-1.139019]], pop: [[40.86098]]
epoch: 11, loss: [[-1.1442757]], pop: [[40.855724]]
epoch: 12, loss: [[-1.1554604]], pop: [[40.84454]]
epoch: 13, loss: [[-1.1703491]], pop: [[40.82965]]
epoch: 14, loss: [[-1.1796303]], pop: [[40.82037]]
epoch: 15, loss: [[-1.1956749]], pop: [[40.804325]]
epoch: 16, loss: [[-1.1605873]], pop: [[40.839413]]
epoch: 17, loss: [[-1.2112846]], pop: [[40.788715]]
epoch: 18, loss: [[-1.2180443]], pop: [[40.781956]]
epoch: 19, loss: [[-1.1957741]

In [9]:
result = np.array(result).reshape(1000, 21)
X_result = scaler.inverse_transform(result)
pop_result = np.array(pop_result).reshape(1000, 1)

In [10]:
df = np.append(X_result, pop_result, axis=1)

In [12]:
pd_df = pd.DataFrame(df)
pd_df.to_excel('./documents/result/basic.xlsx')